In [ ]:
import ipywidgets as widgets
from IPython.display import display
import functions as functions
import importlib


In [ ]:
def load2(b=None):
    use_config = top_pane_dict['buttons'][0].value
    log_path = top_pane_dict['file_choosers'][0].selected
    config_path = top_pane_dict['file_choosers'][1].selected

    log, var_dict, startDict, raceIndex, marksDict, raceEvents, eventLog, selector_dict_day = functions.load_and_process(log_path,config_path,use_config)
    load2.log = log
    load2.var_dict = var_dict
    load2.startDict = startDict
    load2.raceIndex = raceIndex
    load2.marksDict = marksDict
    load2.raceEvents = raceEvents
    load2.eventLog = eventLog
    load2.selector_dict_day = selector_dict_day
    plots = functions.plot_race(startDict, raceEvents, log, marksDict, raceIndex) # create plots for each race
    
    plot_tab = functions.build_plot_tabs(plots)
    
    mid_pane = widgets.VBox([plot_tab,selector_dict_day['formatted']])
    load2.mid_pane = mid_pane
    



In [ ]:
top_pane_dict = functions.build_top_pane()
display(top_pane_dict['formatted'][0])

In [ ]:
load_button = widgets.Button(description='Load and Process Files')
mid_pane_output = widgets.Output()

    
@mid_pane_output.capture()
def load(change):
    load2()
    with mid_pane_output:
        display(load2.mid_pane)
    
    
load_button.on_click(load)
display(load_button)
mid_pane_output

In [ ]:
#filePath_export = top_pane_dict['file_choosers'][0].selected_path
#fileName_export = "test123.xml"
#functions.export_to_xml_worker(load2.selector_dict_day, load2.raceEvents, load2.startDict, load2.log, filePath_export, fileName_export)



